In [2]:
import csv
import geojson
import folium
import pytz
import os

from datetime import datetime
from collections import defaultdict
from matplotlib import pyplot as plt

data_path = '/mnt/c/Users/cephei/Desktop/devicepositions'

# SQL para extraer datos de la base de datos

In [ ]:
# ingresar a la shell de postgres
#python manage.py dbshell

# obtener datos de nearbybuses_log y guardarlos en un csv
#\copy (select * from "AndroidRequests_devicepositionintime" where "timeStamp"> '2018-01-01 00:00:00' order by "timeStamp" ASC) To '/tmp/devicepositions.csv' WITH CSV DELIMITER ',' HEADER;


# Procesar log de posiciones de usuario

In [14]:
device_log_path = os.path.join(data_path, 'devicepositions.csv')

lower_threshold = datetime(2016, 6, 1, tzinfo=pytz.UTC)
upper_threshold = datetime(2019, 6, 20, tzinfo=pytz.UTC)
print('thresholds: {0} - {1}'.format(lower_threshold, upper_threshold))

users = defaultdict(list)
# se almacenan locaciones de un usuario que tengan una distancia temporal superior a time_threshold_in_secs
time_threshold_in_secs = 120
previous_timestamp = defaultdict(lambda: datetime(2000, 1, 1, tzinfo=pytz.UTC))

points_number = 0

with open(device_log_path, 'r') as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',')
    for row in spamreader:
        try:
            timestamp = datetime.strptime(row['timeStamp'] + '00', '%Y-%m-%d %H:%M:%S.%f%z')
        except ValueError:
            timestamp = datetime.strptime(row['timeStamp'] + '00', '%Y-%m-%d %H:%M:%S%z')
        
        if row['phoneId'] != '8ae2ffec-fd2f-4a70-a534-b861c734ec97':
            continue
            
        if lower_threshold < timestamp < upper_threshold and \
           (timestamp - previous_timestamp[row['phoneId']]).total_seconds() > time_threshold_in_secs:
            previous_timestamp[row['phoneId']] = timestamp
            # trunc to hour
            #timestamp = timestamp.replace(hour=0, minute=0, second=0, microsecond=0)

            users[row['phoneId']].append([row['timeStamp'], row['latitude'], row['longitude']])
            points_number += 1

print('user number: {0}'.format(len(users)))
print('points number: {0}'.format(points_number))

thresholds: 2016-06-01 00:00:00+00:00 - 2019-06-20 00:00:00+00:00
user number: 1
points number: 13


# construir archivo para kepler

In [13]:
output_path = os.path.join(data_path, 'salida.csv')

header = ['phone_id', 'timestamp', 'latitude', 'longitude']

stop_error = set()
with open(output_path, 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(header)
    for phone_id in users.keys():
        for position in users[phone_id]:
            spamwriter.writerow([phone_id] + position)

# Ver posición de un usuario

In [7]:
import time

from geojson import LineString, MultiLineString, Feature
from IPython import display

phone_id = ''

for positon in users[phone_id]:
    if True:
        # ====================================
        beauchef_location = [-33.455328, -70.6682527]
        zoom = 11
        m = folium.Map(location=beauchef_location, zoom_start=zoom)
        _shape = LineString(route_shape)
        folium.GeoJson(_shape, name='geojson').add_to(m)

        display.clear_output(wait=True)
        display.display(route_number, row['shape_id'])
        display.display(m)
        time.sleep(0.5)
        # ========================================